[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.leafmap.org/lab/index.html?path=notebooks/78_read_raster.ipynb)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/leafmap/blob/master/docs/notebooks/78_read_raster.ipynb)
[![image](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/opengeos/leafmap/HEAD)

**Interactive Extraction and Visualization of AWS Open Geospatial Data**


Uncomment the following line to install [leafmap](https://leafmap.org) if needed.

In [ ]:
# %pip install -U leafmap

In [ ]:
from leafmap import leafmap

Set custom STAC endpoints.

In [ ]:
catalogs = {
    "Element84 Earth Search": "https://earth-search.aws.element84.com/v1",
    "Microsoft Planetary Computer": "https://planetarycomputer.microsoft.com/api/stac/v1",
}

Display the STAC search GUI. Pan and zoom to the area of interest and use the drawing tools to draw a bounding box or polygon. Select a STAC catalog and click the **Collections** button to retrieve the collections, then click on the **Items** button to retrieve the items within the bounding box or polygon. Select an item from the dropdown list and click the **Display** button to display the item on the map.

In [ ]:
m = leafmap.Map(center=[37.7452, -122.4108], zoom=12, catalog_source=catalogs)
m.add("stac")
m

Display the information of the selected item.

In [ ]:
# m.stac_item

Alternatively, you can search the STAC catalog programmatically by providing a bounding box, time range, and other filters. The example below use the [Earth Search](https://stacindex.org/catalogs/earth-search#/) STAC endpoint by Element 84 for searching for AWS Open Data.

In [ ]:
url = "https://earth-search.aws.element84.com/v1/"
collection = "sentinel-2-l2a"
time_range = "2023-04-01/2023-07-31"
bbox = [-122.491, 37.7208, -122.411, 37.7786]

Search the STAC catalog and return the results as an ItemCollection.

In [ ]:
search = leafmap.stac_search(
    url=url,
    max_items=5,
    collections=[collection],
    bbox=bbox,
    datetime=time_range,
    query={"eo:cloud_cover": {"lt": 10}},
    get_collection=True,
)
# search

Search the STAC catalog and return the results as a dictionary of assets.

In [ ]:
search = leafmap.stac_search(
    url=url,
    max_items=5,
    collections=[collection],
    bbox=bbox,
    datetime=time_range,
    query={"eo:cloud_cover": {"lt": 10}},
    get_assets=True,
)
# search

Get the first item in the collection.

In [ ]:
name, item = next(iter(search.items()))
name

Retrieve the item's assets, which are links to the actual data files.

In [ ]:
item

Retrieve the STAT item's URLs.

In [ ]:
search = leafmap.stac_search(
    url=url,
    max_items=5,
    collections=[collection],
    bbox=bbox,
    datetime=time_range,
    query={"eo:cloud_cover": {"lt": 10}},
    get_links=True,
)
search

Check the band names of the selected item.

In [ ]:
url = search[0]
bands = leafmap.stac_bands(url)
bands[:10]

Display the selected item on the map.

In [ ]:
m = leafmap.Map()
m.add_stac_layer(url, bands=["nir", "red", "green"], name="Sentinel-2")
m

Use the drawing tools to draw a small bounding box on the image.

In [ ]:
if m.user_roi is not None:
    roi = m.user_roi_bounds()
else:
    roi = [-122.5315, 37.6882, -122.3523, 37.8166]

Specify the bands to use.

In [ ]:
bands = ["nir", "red", "green"]

Display the COG URL.

In [ ]:
item["nir"]

Extract one single band within the bounding box as an a numpy array.

In [ ]:
array = leafmap.read_raster(item["nir"], window=roi, coord_crs="epsg:4326")

Check the shape of the array.

In [ ]:
array.shape

Extract multiple bands within the bounding box as an a numpy array.

In [ ]:
sources = [item["nir"], item["red"], item["green"]]
array = leafmap.read_rasters(sources, window=roi, coord_crs="epsg:4326")

Check the shape of the array.

In [ ]:
array.shape

Convert the numpy array to a Cloud Optimized GeoTIFF (COG).

In [ ]:
leafmap.numpy_to_cog(
    array, "s2.tif", bounds=roi, profile=item["nir"], coord_crs="epsg:4326"
)

Display the image on the map.

In [ ]:
m = leafmap.Map()
m.add_raster("s2.tif", band=[1, 2, 3], vmin=0, vmax=4000, layer_name="Subset")
m